## mojo 에서 movie code 가져오기

In [1]:
#어?! 다시왔어?! package
#어?! 다시왔어?! jupyter package
#import
import requests
from bs4 import BeautifulSoup
import time
from random import randint
import re
import pandas as pd
import pickle

#installing selenium, prepare to connect 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

wd = webdriver.Chrome(r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\chromedriver")

In [5]:
#실제 수집

# 수집할 영화 연도 범위
released_years = []
released_months = []

years = [released_years.append(i) for i in range(2015,2020)] #연
months = [released_months.append(i) for i in range(1,13)] #월
Link_urls = []
Movie_code = []

error_urls = [] #movie_code못구하겠다링
movie_title = [] #title summary 안눌려서 영화제목이라도 저장..
#wait
wait = WebDriverWait(wd, 10)

for year in released_years:
  for month in released_months:
    url = "https://www.boxofficemojo.com/calendar/"+str(year)+"-"+str(month)+"-01/" #웹페이지 url
    wd.get(url) #연월별 창 열기
    try:
      wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#table > div > table > tbody >tr > td.a-text-left.mojo-header-column.mojo-truncate.mojo-field-type-release.mojo-cell-wide > div >  a.a-link-normal")))
    except TimeoutException:
      wd.get(url) #
      time.sleep(randint(2,4))
    #기존links = wd.find_elements_by_css_selector("#table > div > table > tbody >tr > td.a-text-left.mojo-header-column.mojo-truncate.mojo-field-type-release.mojo-cell-wide")
    links = wd.find_elements_by_css_selector("#table > div > table > tbody >tr")
    #for movie_link in links:
    for link_ind in range(len(links)+1): #해당 월에 몇개 영화데이터있나 숫자만 가져오기
      #nth child로 몇번째 클릭할지 지정해주자
      try:
        movie_tag = "#table > div > table > tbody > tr:nth-child("+str(link_ind)+") > td.a-text-left.mojo-header-column.mojo-truncate.mojo-field-type-release.mojo-cell-wide > div >  a.a-link-normal"
        Movie_link = wd.find_element_by_css_selector(movie_tag)
        Movie_link.click() #클릭하는 순간 이전의 세션에 생성한 links 사용 불가. 
        
      except NoSuchElementException:
        #print(link_ind,"는 비어있지롱 >< ")
        continue   
      #mojo개별 페이지 진입됨.
      try: 
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#title-summary-refiner > a > span")))#개별영화 페이지 기다리기
      except TimeoutException: #느려서 titlesummary가 클릭 안될 때.
        time.sleep(randint(2,4))
        
      try:
        title_summary = wd.find_element_by_css_selector("#title-summary-refiner > a > span")
        title_summary.click()
        time.sleep(randint(1,3))  
        mojo_url = wd.current_url
      except: #영화코드 못구하면 타이틀이라도
        title = wd.find_element_by_css_selector("#a-page > main > div > div.a-section.a-spacing-none.mojo-summary > div.a-section.mojo-heading-summary > div > div > div.a-fixed-left-grid-col.a-col-right > h1")
        movie_title.append(title)
      # 4. movie_code 수집
      target = str(mojo_url)
      try:
        p = re.compile(r"(https://www.boxofficemojo.com/title/)([a-zA-z]{2}[0-9]{7})(/.)")
        m = p.match(target)
        movie_code = m.group(2) #영화코드수집 --완료
      except AttributeError:
          error_url = wd.current_url
          print(error_url,"group이 또 정규표현식 말썽이네이거이거")
      except:
        error_urls.append(wd.current_url)
        print(error_url,"group은 아닌데 오류 오류")
      Movie_code.append(movie_code)
      wd.close
      try:
        wd.get(url)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#table > div > table > tbody >tr > td.a-text-left.mojo-header-column.mojo-truncate.mojo-field-type-release.mojo-cell-wide > div >  a.a-link-normal")))
      except TimeoutException:
        wd.get(url)
        time.sleep(randint(2,4))
    wd.close #연월별창 닫기
  #총 mojo 숫자와 비교.      
  print("기존 Mojo 영화 갯수:16014","Mojo새코드:",len(Movie_code))
  ypickle_name = "Movie_code"+str(year)+".txt"
  with open (ypickle_name,"wb") as f:#연별로 피클저장
    pickle.dump(Movie_code,f)
#피클에 담기
with open ("Movie_code.txt","wb") as f:
  pickle.dump(Movie_code,f)



기존 Mojo 영화 갯수:16014 Mojo새코드: 967
기존 Mojo 영화 갯수:16014 Mojo새코드: 1989
https://www.boxofficemojo.com/title/tt10192670/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
기존 Mojo 영화 갯수:16014 Mojo새코드: 3076
https://www.boxofficemojo.com/title/tt10192640/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt11376890/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt11376890/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt10192672/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt10192676/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt10160730/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt10160700/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt10160686/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt10160782/?ref_=bo_rl_ti group이 또 정규표현식 말썽이네이거이거
https://www.boxofficemojo.com/title/tt1

In [9]:
ypickle_name = "Movie_code"+str(year)+".txt"
with open (ypickle_name,"wb") as f:#연별로 피클저장
  pickle.dump(Movie_code,f)
